In [1]:
from pybatfish.client.commands import *
from pybatfish.datamodel import *
from pybatfish.question import bfq, load_questions

load_questions()

Successfully loaded 55 questions from remote


In [5]:
from pybatfish.client.capirca import create_reference_book, init_snapshot_from_acl

In [7]:
b = create_reference_book('capirca/defs/')

In [8]:
b

ReferenceBook(name='capirca', addressGroups=[AddressGroup(name='HOST_BITS', addresses=['1.0.0.0/8'], childGroupNames=[]), AddressGroup(name='RFC1918_10', addresses=['10.0.0.0/8'], childGroupNames=[]), AddressGroup(name='RFC1918_172', addresses=['172.16.0.0/12'], childGroupNames=[]), AddressGroup(name='RFC1918_192', addresses=['192.168.0.0/16'], childGroupNames=[]), AddressGroup(name='RFC1918', addresses=[], childGroupNames=['RFC1918_10', 'RFC1918_172', 'RFC1918_192']), AddressGroup(name='LOOPBACK', addresses=['127.0.0.0/8'], childGroupNames=[]), AddressGroup(name='RFC_5737_DOCUMENTATION_BLOCKS', addresses=['192.0.2.0/24', '198.51.100.0/24', '203.0.113.0/24'], childGroupNames=[]), AddressGroup(name='BGP_SPEAKERS', addresses=['203.0.113.16/28'], childGroupNames=[]), AddressGroup(name='INTERNAL_SERVERS', addresses=['203.0.113.36/30'], childGroupNames=[]), AddressGroup(name='MAIL_SERVERS', addresses=['203.0.113.32/29'], childGroupNames=[]), AddressGroup(name='OUTLOOK_SERVERS', addresses=['203.0.113.32/30'], childGroupNames=[]), AddressGroup(name='WEB_SERVERS', addresses=['203.0.113.0/24'], childGroupNames=[])], interfaceGroups=[])

In [ ]:
from pybatfish.client.extended import *
print(bf_get_snapshot_input_object_text('configs/config', snapshot=juniper_border))

In [ ]:
# 3+ Now do further, comprehensive ACL checks using Batfish.
bfq.filterLineReachability().answer(juniper_border)

In [ ]:
# Assert equivalence between Cisco and Juniper capirca renderings (no results)
bfq.searchFilters().answer(arista_border, juniper_border)

In [ ]:
from pybatfish.client.capirca import create_reference_book
b = create_reference_book(DEFS_DIR)
bf_put_reference_book(b)

In [ ]:
from pybatfish.client.asserts import *

In [ ]:
TEST_SNAPSHOT=juniper_border

# Allow incoming BGP sessions to be established
assert_filter_permits(
    filter_name='border',
    headers=HeaderConstraints(
        srcIps='@addressGroup(capirca, bgp_speakers)',
        dstIps='@addressGroup(capirca, bgp_speakers)',
        ipProtocols='tcp',
        dstPorts=179),
    snapshot=TEST_SNAPSHOT
)

# Allow reverse traffic for outgoing BGP sessions
assert_filter_permits(
    filter_name='border',
    headers=HeaderConstraints(
        srcIps='@addressGroup(capirca, bgp_speakers)',
        dstIps='@addressGroup(capirca, bgp_speakers)',
        ipProtocols='tcp',
        srcPorts=179),
    snapshot=TEST_SNAPSHOT
)

# Deny incoming BGP sessions to net_mtbb that are not from net_mtbb_peers
assert_filter_denies(
    filter_name='border',
    headers=HeaderConstraints(
        srcIps='0.0.0.0/0 \ @addressGroup(capirca, bgp_speakers)',
        dstIps='@addressGroup(capirca, bgp_speakers)',
        ipProtocols='tcp',
        dstPorts=179),
    snapshot=TEST_SNAPSHOT
)

# Deny incoming new BGP sessions to everyone that is not an MTBB peer
assert_filter_denies(
    filter_name='border',
    headers=HeaderConstraints(
        srcIps='0.0.0.0/0',
        dstIps='0.0.0.0/0 \ @addressGroup(capirca, bgp_speakers)',
        tcpFlags=MatchTcpFlags(tcpFlags=TcpFlags(syn=True, ack=False, rst=False), useSyn=True, useAck=True, useRst=True),
        ipProtocols='tcp',
        dstPorts=179),
    snapshot=TEST_SNAPSHOT
)